In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.regressionplots import influence_plot
import statsmodels.formula.api as smf
import numpy as np

In [ ]:
Startups = pd.read_csv(r"D:\Data Science\Jupyter Notebook\CSV files\50_Startups.csv")
Startups.head()

In [ ]:
Startups.drop(Startups.columns[3], axis = 1, inplace = True)
Startups= Startups.rename(columns={'R&D Spend':'RD_Spend','Marketing Spend':'Marketing_Spend'})
Startups.head()

In [ ]:
Startups.info()

In [ ]:
#check for missing values
Startups.isna().sum()

In [ ]:
# Correlation Matrix
Startups.corr()

In [ ]:
# Scatterplot b/w variables along with histograms
sns.set_style(style='darkgrid')
sns.pairplot(Startups)

In [ ]:
# Building a model
import statsmodels.formula.api as smf 
model = smf.ols('Profit~RD_Spend+Administration+Marketing_Spend',data=Startups).fit()
model

In [ ]:
#Coefficients
model.params

In [ ]:
# t-values and p-values
model.tvalues,np.round(model.pvalues,5)

In [ ]:
#R squared values
(model.rsquared,model.rsquared_adj)

In [ ]:
# Build SLR and MLR models for insignificant variables 'Administration' and 'Marketing_Spend'
# Also find their t-values and p-values

In [ ]:
# Simple Linear Regression Models
# t-values and p-Values
slr_a=smf.ols('Profit~Administration',data = Startups).fit()
slr_a.tvalues, slr_a.pvalues # Administration has in-significant p-value 

In [ ]:
slr_m=smf.ols("Profit~Marketing_Spend",data=Startups).fit()
slr_m.tvalues , slr_m.pvalues  # Marketing_Spend has significant p-value

In [ ]:
mlr_am=smf.ols("Profit~Administration+Marketing_Spend",data=Startups).fit()
mlr_am.tvalues , mlr_am.pvalues  # varaibles have significant p-values

In [ ]:
# Model Validation
# Two Techniques: 1. Collinearity Check & 2. Residual Analysis
# Collinearity Check, Calculating VIF
rsq_Administration = smf.ols('Administration~RD_Spend+Marketing_Spend',data=Startups).fit().rsquared  
vif_Administration = 1/(1-rsq_Administration) 
vif_Administration

In [ ]:
rsq_RD_Spend = smf.ols('RD_Spend~Administration+Marketing_Spend',data=Startups).fit().rsquared  
vif_RD_Spend = 1/(1-rsq_RD_Spend)
vif_RD_Spend

In [ ]:
rsq_Marketing_Spend = smf.ols('Marketing_Spend~RD_Spend+Administration',data=Startups).fit().rsquared  
vif_Marketing_Spend = 1/(1-rsq_Marketing_Spend)  #564.84
vif_Marketing_Spend

In [ ]:
# Storing VIF values in a data frame
d1 = {'Variables':['RD_Spend','Administration','Marketing_Spend'],'VIF':[vif_RD_Spend,vif_Administration,vif_Marketing_Spend]}
Vif_frame = pd.DataFrame(d1)  
Vif_frame

None variable has VIF>20, No Collinearity, so consider all varaibles in Regression equation

In [ ]:
# Residual Analysis
# Test for Normality of Residuals (Q-Q Plot)
import statsmodels.api as sm
qqplot=sm.qqplot(model.resid,line='q') # line = q to draw the diagnoal line
plt.title("Normal Q-Q plot of residuals")
plt.show()

In [ ]:
list(np.where(model.resid<-30000)) 

In [ ]:
#Residual Plot for Homoscedasticity
def get_standardized_values( vals ):
    return (vals - vals.mean())/vals.std()   # User defined z = (x - mu)/sigma

In [ ]:
plt.scatter(get_standardized_values(model.fittedvalues),get_standardized_values(model.resid))
plt.title('Residual Plot')
plt.xlabel('Standardized Fitted values')
plt.ylabel('Standardized residual values')
plt.show()

In [ ]:
# Residual vs Regressors
# Test for errors or Residuals Vs Regressors or independent 'x' variables or predictors 
# using Residual Regression Plots code graphics.plot_regress_exog(model,'x',fig)    # exog = x-variable & endog = y-variable
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "Administration", fig=fig)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "RD_Spend", fig=fig)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "Marketing_Spend", fig=fig)
plt.show()

In [ ]:
# Model Deletion Diagnostics
# Detecting Influencers/Outliers
# Cook's Distance
model_influence = model.get_influence()
(c, _) = model_influence.cooks_distance
c

In [ ]:
# Plot the influencers values using stem plot
fig = plt.subplots(figsize=(20, 7))
plt.stem(np.arange(len(Startups)), np.round(c, 5))
plt.xlabel('Row index')
plt.ylabel('Cooks Distance')
plt.show()

In [ ]:
# Index and value of influencer where c is more than 0.5
(np.argmax(c),np.max(c))

In [ ]:
# High Influence points
from statsmodels.graphics.regressionplots import influence_plot
influence_plot(model)
plt.show()

In [ ]:
# Leverage Cuttoff Value = 3*(k+1)/n ; k = no.of features/columns & n = no. of datapoints
k = Startups.shape[1]
n = Startups.shape[0]
leverage_cutoff = (3*(k+1))/n
leverage_cutoff

In [ ]:
Startups[Startups.index.isin([49])] 

From the above plot, it is evident that data point 49 is influencer

In [ ]:
# Improving the model
# Discard the data points which are influencers and reassign the row number (reset_index(drop=True))
Startups2=Startups.drop(Startups.index[[49]],axis=0).reset_index(drop=True)
Startups2

In [ ]:
model2=smf.ols("Profit~RD_Spend+Administration+Marketing_Spend",data=Startups2).fit()

In [ ]:
# Build Model
# Model Deletion Diagnostics and Final Model
while model2.rsquared < 0.99:
    for c in [np.max(c)>1]:
        model2=smf.ols("Profit~RD_Spend+Administration+Marketing_Spend",data=Startups2).fit()
        (c,_)=model2.get_influence().cooks_distance
        c
        np.argmax(c) , np.max(c)
        Startups2=Startups2.drop(Startups2.index[[np.argmax(c)]],axis=0).reset_index(drop=True)
        Startups2
    else:
        final_model=smf.ols("Profit~RD_Spend+Administration+Marketing_Spend",data=Startups2).fit()
        final_model.rsquared , final_model.aic

In [ ]:
final_model.rsquared

In [ ]:
Startups2

In [ ]:
# Model Predictions
# say New data for prediction is
new_data=pd.DataFrame({'RD_Spend':70000,"Administration":90000,"Marketing_Spend":140000},index=[0])
new_data

In [ ]:
# Manual Prediction of Price
final_model.predict(new_data)

In [ ]:
# Automatic Prediction of Price with 90.02% accurcy
pred_y=final_model.predict(Startups2)
pred_y

In [ ]:
# table containing R^2 value for each prepared model
d2={'Prep_Models':['Model','Final_Model'],'Rsquared':[model.rsquared,final_model.rsquared]}
table=pd.DataFrame(d2)
table